<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Hands-on Lab: Complete the Data Collection with Web Scraping lab


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`:
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests
!pip install pandas

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()

    colunm_name = ' '.join(row.contents)

    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [11]:
# use requests.get() method with the provided static_url
# assign the response to a object

import pandas as pd
import requests

# Wikipedia URL
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Add User-Agent header to avoid HTTPError
headers = {
    "User-Agent": "Mozilla/5.0"
}

# Request page content
response = requests.get(static_url, headers=headers)
response.raise_for_status()  # will raise error if request fails

# Read HTML tables
html_tables = pd.read_html(response.text)

# Extract the third table
first_launch_table = html_tables[2]

# Display table
first_launch_table.head()


/tmp/ipython-input-2655933152.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  html_tables = pd.read_html(response.text)


,Flight No.,Date and time (UTC),"Version, Booster [b]",Launch site,Payload[c],Payload mass,Orbit,Customer,Launch outcome,Booster landing
0,1,"4 June 2010, 18:45",F9 v1.0[7] B0003.1[8],"CCAFS, SLC-40",Dragon Spacecraft Qualification Unit,NaN,LEO,SpaceX,Success,Failure[9][10] (parachute)
1,1,First flight of Falcon 9 v1.0.[11] Used a boil...,First flight of Falcon 9 v1.0.[11] Used a boil...,First flight of Falcon 9 v1.0.[11] Used a boil...,First flight of Falcon 9 v1.0.[11] Used a boil...,First flight of Falcon 9 v1.0.[11] Used a boil...,First flight of Falcon 9 v1.0.[11] Used a boil...,First flight of Falcon 9 v1.0.[11] Used a boil...,First flight of Falcon 9 v1.0.[11] Used a boil...,First flight of Falcon 9 v1.0.[11] Used a boil...
2,2,"8 December 2010, 15:43[13]",F9 v1.0[7] B0004.1[8],"CCAFS, SLC-40",Dragon demo flight C1 (Dragon C101),NaN,LEO (ISS),NASA (COTS) NRO,Success[9],Failure[9][14] (parachute)
3,2,"Maiden flight of Dragon capsule, consisting of...","Maiden flight of Dragon capsule, consisting of...","Maiden flight of Dragon capsule, consisting of...","Maiden flight of Dragon capsule, consisting of...","Maiden flight of Dragon capsule, consisting of...","Maiden flight of Dragon capsule, consisting of...","Maiden flight of Dragon capsule, consisting of...","Maiden flight of Dragon capsule, consisting of...","Maiden flight of Dragon capsule, consisting of..."
4,3,"22 May 2012, 07:44[17]",F9 v1.0[7] B0005.1[8],"CCAFS, SLC-40",Dragon demo flight C2+[18] (Dragon C102),"525 kg (1,157 lb)[19]",LEO (ISS),NASA (COTS),Success[20],No attempt


Create a `BeautifulSoup` object from the HTML `response`


In [6]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content

Print the page title to verify if the `BeautifulSoup` object was created properly


In [7]:
# Use soup.title attribute

### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [8]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`


Starting from the third table is our target table contains the actual launch records.


In [12]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

    Flight No.                                Date and time (UTC)  \
0            1                                 4 June 2010, 18:45   
1            1  First flight of Falcon 9 v1.0.[11] Used a boil...   
2            2                         8 December 2010, 15:43[13]   
3            2  Maiden flight of Dragon capsule, consisting of...   
4            3                             22 May 2012, 07:44[17]   
5            3  Dragon spacecraft demonstrated a series of tes...   
6            4                          8 October 2012, 00:35[21]   
7            4                          8 October 2012, 00:35[21]   
8            4  CRS-1 was successful, but the secondary payloa...   
9            5                                1 March 2013, 15:10   
10           5  Last launch of the original Falcon 9 v1.0 laun...   
11           6                       29 September 2013, 16:00[30]   
12           6  First commercial mission with a private custom...   
13           7                    

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [13]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names


Check the extracted column names


In [14]:
print(column_names)

[]


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [18]:
launch_dict= dict.fromkeys(column_names)

launch_dict = dict.fromkeys(column_names)

# Safely remove Date and time column
for key in list(launch_dict.keys()):
    if 'Date and time' in key:
        del launch_dict[key]

# Initialize remaining columns
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []

# Added new columns
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [22]:
# =============================================================
# COMPLETE WIKIPEDIA SCRAPER – ONE CELL
# =============================================================

import requests
from bs4 import BeautifulSoup
import pandas as pd

# -------------------------------------------------------------
# Helper functions (as used in IBM SpaceX lab)
# -------------------------------------------------------------

def date_time(cell):
    datetime_list = list(cell.stripped_strings)
    return datetime_list

def booster_version(cell):
    links = cell.find_all('a')
    return links[0].string if links else None

def get_mass(cell):
    mass = ''.join(cell.stripped_strings)
    if mass == '':
        return None
    return mass.replace(',', '')

def landing_status(cell):
    return " ".join(cell.stripped_strings)

# -------------------------------------------------------------
# Wikipedia URL
# -------------------------------------------------------------
static_url = (
    "https://en.wikipedia.org/w/index.php?"
    "title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
)

# -------------------------------------------------------------
# Request page
# -------------------------------------------------------------
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(static_url, headers=headers)
response.raise_for_status()

# -------------------------------------------------------------
# Create soup
# -------------------------------------------------------------
soup = BeautifulSoup(response.text, "html.parser")

# -------------------------------------------------------------
# Initialize launch dictionary
# -------------------------------------------------------------
launch_dict = {
    'Flight No.': [],
    'Launch site': [],
    'Payload': [],
    'Payload mass': [],
    'Orbit': [],
    'Customer': [],
    'Launch outcome': [],
    'Version Booster': [],
    'Booster landing': [],
    'Date': [],
    'Time': []
}

# -------------------------------------------------------------
# Extract data
# -------------------------------------------------------------
extracted_row = 0

for table in soup.find_all('table', "wikitable plainrowheaders collapsible"):

    for rows in table.find_all("tr"):

        # Check flight number
        if rows.th and rows.th.string:
            flight_number = rows.th.string.strip()
            flag = flight_number.isdigit()
        else:
            flag = False

        row = rows.find_all("td")

        if flag and len(row) >= 9:
            extracted_row += 1

            # Date & Time
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',') if len(datatimelist) > 0 else None
            time = datatimelist[1] if len(datatimelist) > 1 else None

            # Booster Version
            bv = booster_version(row[1])
            if not bv:
                bv = " ".join(row[1].stripped_strings)

            # Safe text extraction (NO AttributeErrors)
            launch_site = " ".join(row[2].stripped_strings)
            payload = " ".join(row[3].stripped_strings)
            payload_mass = get_mass(row[4])
            orbit = " ".join(row[5].stripped_strings)
            customer = " ".join(row[6].stripped_strings)
            launch_outcome = " ".join(row[7].stripped_strings)
            booster_landing = landing_status(row[8])

            # Append to dictionary
            launch_dict['Flight No.'].append(flight_number)
            launch_dict['Launch site'].append(launch_site)
            launch_dict['Payload'].append(payload)
            launch_dict['Payload mass'].append(payload_mass)
            launch_dict['Orbit'].append(orbit)
            launch_dict['Customer'].append(customer)
            launch_dict['Launch outcome'].append(launch_outcome)
            launch_dict['Version Booster'].append(bv)
            launch_dict['Booster landing'].append(booster_landing)
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)

# -------------------------------------------------------------
# Create DataFrame
# -------------------------------------------------------------
launch_df = pd.DataFrame(launch_dict)

print("Total rows extracted:", extracted_row)
launch_df.head()


Total rows extracted: 121


,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,"CCAFS , SLC-40",Dragon Spacecraft Qualification Unit,None,LEO,SpaceX,Success,F9 v1.0,Failure [ 9 ] [ 10 ] (parachute),4 June 2010,18:45
1,2,"CCAFS , SLC-40",Dragon demo flight C1 (Dragon C101),None,LEO ( ISS ),NASA ( COTS ) NRO,Success [ 9 ],F9 v1.0,Failure [ 9 ] [ 14 ] (parachute),8 December 2010,15:43
2,3,"CCAFS , SLC-40",Dragon demo flight C2+ [ 18 ] (Dragon C102),525 kg (1157 lb)[19],LEO ( ISS ),NASA ( COTS ),Success [ 20 ],F9 v1.0,No attempt,22 May 2012,07:44
3,4,"CCAFS , SLC-40",SpaceX CRS-1 [ 22 ] (Dragon C103),4700 kg (10400 lb),LEO ( ISS ),NASA ( CRS ),Success,F9 v1.0,No attempt,8 October 2012,00:35
4,5,"CCAFS , SLC-40",SpaceX CRS-2 [ 22 ] (Dragon C104),4877 kg (10752 lb),LEO ( ISS ),NASA ( CRS ),Success,F9 v1.0,No attempt,1 March 2013,15:10


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [23]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab.

Following labs will be using a provided dataset to make each lab independent.


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
